# Project description "Collaboration and Competition"

The "Collaboration and Competition" project is the final problem in [Udacity](https://www.udacity.com/)'s Deep Reinforcement Learning Nanodegree. A description of the environment can be found in [0], the information in the following paragraphs is from that source: The task of the project is to teach two agents to control tennis rackets in order to play tennis together. Each time an agent hits a ball and it passes the net the agent gets a reward of + 0.1, each time the ball hits the ground the agent gets a reward of -0.01. At the end of an episode the cumulative score for each agent is calculated and the maximum of these two values is taken as the total score for this episode. The task is solved when the average score over 100 episodes is above 0.5.

Each agent has a 24 dimensional observation space: 8 numbers describe the positions and velocities of ball and the racket at one time step, and information from three time steps is saved for a single observation. The action space consists of two continuous actions: one for the horizontal movement, one to control the vertical movement.

This task needs cooperation between two agents as a base the individual agents I used [my implementation](https://github.com/pluebcke/drlnd-continuous_control) of the Continuous Control project. The base algorithm for the individual agents is the the Twin Delayed DDPG algorithm (TD3, Fujimoto et al., 2018, [1]) which has some improvements over DDPG [2]. In order to enable the two agents learn a cooperative policy, a set-up similar to multi-agent deep deterministic policy gradient (MADDPG) by Lowe et al.(2017, [3]) was used.

## Actor Critic Methods, DDPG & T3D algorithm
### DDPG &TD3
Before explaining the details of the multi-agent set-up, the theory behind DDPG and TD3 from my [Continuous Control project report](https://github.com/pluebcke/drlnd-continuous_control) is repeated here:

Techniques like DQN learning work well with discrete and relatively small action spaces, however in continuous action spaces, finding the optimal value of the Q-value function is an optimization problem on its own [2]. Lillicrap et al. (2015, [2]) solved this problem by using an actor-critic method. In the actor-critic method a policy (actor) is directly learned with help of the Q-value function (critic). 

While the original manuscripts are definitely worth reading, I will briefly mention the main ideas following the summaries given by "Spinning Up Deep RL" (see [4] for their DDPG description and [5] for TD3):

Deep Deterministic Policy Gradient learns a representation $\phi_{local}$ of the action-value function Q very similar to the DQN algorithm by using the Bellman equation. In order to improve stability another representation of the action value function $Q_{target}$ is used, and this neural network is slowly updated to match $Q_{local}$. The target action-value function $Q_{target}$ can be updated with help of the the Bellman equation:

$$Q_{target}(s_{0}, a_{0})=r + \gamma * Q_{target}(s_{1}, \mu_{\phi_{target}}(s_{1}))$$ 
where $s_{0}$ is the current state, $a_{0}$ the current action, r the reward after taking the action and $s_{1}$ is the next sate.

The local network is then updated by minimizing the mean squared error between $Q_{target}(s_{0}, a_{0})$ and $Q_{local}(s_{0}, a_{0})$.

The representation of the policy $\mu_{\theta}$ is learned by taking gradient ascent steps in the direction of the gradient of the expected action-value function [2]:
$$ \nabla_{\theta}Q_{\phi}(s_{0}, \mu_{\theta}(s_{0}))$$
As explained in [4] and [5] DDPG can be instable due to an overestimation of the learned Q-values. 
The T3D algorithm by Fujimoto et al. (2018, [2]) adds three improvements to the DDPG method that help stabilize the algorithm's performance:
- The algorithm uses two Q-functions $Q_{A}$ and $Q_{B}$. The minimum of the predictions of the Q-value, $min(Q_{A}\mu_{\phi_{target}}(s_{1}), Q_{B}\mu_{\phi_{target}}(s_{1}))$ is used during the Bellman update of the critic neural network. This helps prevent overestimation of the Q-value function during training.
- The actor (or policy) network is updated less frequently than the critic networks
- Noise is added to the target action in the critic update step, which helps, for example, prevent problems due to spikes in the learned Q-value function.

### Multi-Agent TD3
In this problem the two agents need to collaborate in order to achieve a large score. One problem when training multiple agents is that their policies change, thus leading to a non-stationary environment for each individual agent. MADDPG by Lowe et al. (2017, [3]) introduces a centralized critic that prevents problems with non-stationarity.
The critic of an agent is trained not only with the agent's observations and actions but with the observations and actions of all agents [3]:

$$ Q(s^{i}_{t},..., s^{n}_{t}, a_{0},...,a_{n})$$

This results in a stationary environment for the agents. 
The input for the neural network for the critic is therefore the combination of all observations and actions. 
The actor, however, only uses the individual observations for each agent, it is therefore able to use the policy without communication between the agents.

## Implementation

I started by using my [Continuous Control project implementation](https://github.com/pluebcke/drlnd-continuous_control)
which loosely follows the [DDPG pendulum implementation by Udacity](https://github.com/udacity/deep-reinforcement-learning/tree/master/ddpg-pendulum) [8]. An "episode generator" is used to produce experience, following an idea from [9]. The improvements of the TD3 algorithm ([1], [5]) are used in the agents: double Q-learning, delayed policy updates and target policy smoothing. Similar to the [original author's implementation](https://github.com/sfujim/TD3) ([6]), I implemented the two critic networks in a single class that returns Q values for both in one step.

The two agents that should learn to cooperate are of type T3DAgent. They are members of the MATD3 class which coordinates playing episodes as well as the learning steps. The main difficulty was the handling of experience (tuples of: states, actions, next_state, reward, done) which is now performed in the learn method. 

In the learning step the prediction of a0 is calculated for all agents, not only for the agent whose policy is updated, similar to [10]. This is different from the algorithm given in Lowe et al. (2017, [3]), who only uses the prediction for the current agent and uses the actually observed actions otherwise. I sticked with this way of calculating the predictions since it resulted in cleaner code and was the first version of the agent that was able to pick up learning. 

The agents are trained significantly more often than in the Continuous Control project. I used 5 learning steps every step in the environment following a suggestion from [11] and [12].



## Results

An animation of the trained agents is shown below:

![Tennis](results/tennis.gif)

I will briefly discuss two sets of results from different settings. More results from different runs can be found in the results folder. 

### First settings:
The settings for the first presented run are shown below, they are similar to the settings of the Continuous Control project with the following changes: 
The batch size and action noise values are larger. In particular the larger action noise seemed to be important to get the agents learning. An action noise of 1.0 with a decay rate of 0.999 was chosen, these settings were also used by [14]. Action noise is decreased at the end of an episode. In earlier runs, with a faster action noise decay, learning seemed to fail if action noise was too small before the agents had learned to pass the ball over the net more than once.
The target action noise as well as the clipping value were chosen slightly larger as well.

In [ ]:
settings1 = {
    'batch_size': 128,           # Number of experience samples per training step
    'buffer_size': int(3e6),     # Max number of samples in the replay memory
    'gamma': 0.99,               # Reward decay factor
    'tau': 1e-3,                 # Update rate for the slow update of the target networks
    'lr_actor': 5e-4,            # Actor learning rate
    'lr_critic': 5e-4,           # Critic learning rate
    'action_noise': 1.0,         # Noise added during episodes played
    'action_noise_decay': 0.999, # Decay of action noise
    'action_noise_min': 0.15,    # minimum value of action noise
    'action_clip': 1.0,          # Actions are clipped to +/- action_clip
    'target_action_noise': 0.6,  # Noise added during the critic update step
    'target_noise_clip': 0.3,    # Noise clip for the critic update step
    'number_steps': 1,           # Number of steps for roll-out, currently not used
    'optimize_actor_every': 2,   # Update the actor only every X update steps
    'pretrain_steps': int(10000), # Number of random actions played before training starts
    'actor_settings': actor_settings,
    'critic_settings': critic_settings}

The learning curve is shown below. The agent clearly picks ups learning after approximately 400 episodes. After 1000 episodes learning seems to accelerate and the agent stops with an average above 1.0 after roughly 1200 episodes.
![Learning curve](results/run000/learning_curve.png)

The actor loss shown below increases steadily throughout training. Since the actor loss is the expected Q value this is a good sign for a learning agent.
![Actor Loss](results/run000/a_loss.png)

The critic loss decreases in the beginning and then increases again for both critic networks. This is probably due to unknown, larger values (i.e. better results) as the agents pick up learning. It would be interesting to see if the critic loss decreases again in the long run.
![Critic Loss](results/run000/c_loss.png)

### Second settings:
Besides this succesful run I also tried more agressive settings which I had previously seen in the implementations [12] and [14].
The learning rate for both actor and critic were increased to 1e-3 as in [12]. The rate for the slow network update $\tau$ was increased to 1e-2.
Following [13] the batch size was increased to 256. The settings for the various noise sources were not changed between these two runs.

In [ ]:
settings2 = {
    'batch_size': 256,            # Number of experience samples per training step
    'buffer_size': int(1e7),      # Max number of samples in the replay memory
    'gamma': 0.99,                # Reward decay factor
    'tau': 1e-2,                  # Update rate for the slow update of the target networks
    'lr_actor': 1e-3,             # Actor learning rate
    'lr_critic': 1e-3,            # Critic learning rate
    'action_noise': 1.0,          # Noise added during episodes played
    'action_noise_decay': 0.999,  # Decay of action noise
    'action_noise_min': 0.10,     # minimum value of action noise
    'action_clip': 1.0,           # Actions are clipped to +/- action_clip
    'target_action_noise': 0.6,   # Noise added during the critic update step
    'target_noise_clip': 0.3,     # Noise clip for the critic update step
    'number_steps': 1,            # Number of steps for roll-out, currently not used
    'optimize_actor_every': 2,    # Update the actor only every X update steps
    'pretrain_steps': int(10000), # Number of random actions played before training starts
    'actor_settings': actor_settings,
    'critic_settings': critic_settings}

The learning curve for the second set of settings is shown below. The agent appears start learning much earlier than in the first set of parameters. It finished with an average score above 1.0 (over 100 episodes) after 843 episodes instead of 1172 episodes.
![Learning curve](results/run001/learning_curve.png)


## Future improvements

Since I only tried choosing the parameters using an "educated guess" combined with values that I had seen in other sources before, it would be a good idea to perform a proper search for a good set of parameters. This was so far not done due to limited computational power.

It would also be interesting to implement more algorithms. As has been said before, Distributed Distributional Deterministic Policy Gradients (D4PG, [6]) would be an interesting candidate as well the many other algorithms that I have not tried to implement yet (Trust Region Policy Optimization, [16] or Proximal Policy Optimization [17] just two name two algorithms that are also featured on OpenAi Spinning Up)

## Further reading and references

Since I took my implementation of the Continuous Control project as a baseline for the multi-agent implementation, I repeat most of the reading list from that project here and add some sources that I read before / while implementing this project.
- As suggested by Udacity, I started by studying the DDPG pendulum implementation [7]
- The descriptions of DDPG and TD3 from OpenAi ([4], [5]) were great to understand both algorithms in more detail
- The TD3 implementation of the original authors can be found under [6]
- The Deep Reinforcement Learning Hands-On book by Maxim Lapan [8] was a valuable source that greatly helped me understand the topic better and the hands-on code examples were really helpful to see, how things can be implemented in Python and PyTorch.
- One Udacity student helped many people in the internal discussion boards and shared his github profile (which you can find at [10]). The github profile features very thorough implementations of all the projects in the DLRND. The idea of using a pre-train phase with random actions before training is from his implementation of Project 2. In this project I tried to use predictions for the entire actor update (rather than just the prediction for the current agent) after seeing this in this implementation of Project 3.
- I read two other blogposts about this project: [11] (with the corresponding github project can be found at [12]) and [13] (with the github project [14]). I would not have tried out as many update cycles without reading these posts. Also the parameters from these two posts converged quicker than the original parameters that I had chosen in my Continuous Control project.
- The PyTorch tutorials and documentation were again very helpful [15] to find commands in PyTorch.

# References
[0] https://github.com/udacity/deep-reinforcement-learning/tree/master/p3_collab-compet  
[1] Fujimoto, Scott, Herke van Hoof, and David Meger. "Addressing function approximation error in actor-critic methods." arXiv preprint arXiv:1802.09477 (2018).  
[2] Lillicrap, Timothy P., et al. "Continuous control with deep reinforcement learning." arXiv preprint arXiv:1509.02971 (2015).  
[3] Lowe, Ryan, et al. "Multi-agent actor-critic for mixed cooperative-competitive environments." Advances in Neural Information Processing Systems. 2017.  
[4] https://spinningup.openai.com/en/latest/algorithms/ddpg.html  
[5] https://spinningup.openai.com/en/latest/algorithms/td3.html  
[6] https://github.com/sfujim/TD3, last accessed: 2019-04-14  
[7] Barth-Maron, Gabriel, et al. "Distributed distributional deterministic policy gradients." arXiv preprint arXiv:1804.08617 (2018).  
[8] https://github.com/udacity/deep-reinforcement-learning/tree/master/ddpg-pendulum, last accessed: 2019-04-14  
[9] Lapan, Maxim. Deep Reinforcement Learning Hands-On: Apply modern RL methods, with deep Q-networks, value iteration, policy gradients, TRPO, AlphaGo Zero and more. Packt Publishing Ltd, 2018.  
[10] https://github.com/whiterabbitobj/Collaborate_Compete, last accessed: 2019-04-21  
[11] https://towardsdatascience.com/training-two-agents-to-play-tennis-8285ebfaec5f, last accessed: 2019-04-22  
[12] https://github.com/tommytracey/DeepRL-P3-Collaboration-Competition, last accessed: 2019-04-22  
[13] https://medium.com/@amitpatel.gt/maddpg-91caa221d75e, last accessed: 2019-04-22  
[14] https://github.com/gtg162y/DRLND/tree/master/P3_Collab_Compete, last accessed: 2019-04-22  
[15] https://pytorch.org/  
[16] Schulman, John, et al. "Trust region policy optimization." International Conference on Machine Learning. 2015.  
[17] Schulman, John, et al. "Proximal policy optimization algorithms." arXiv preprint arXiv:1707.06347 (2017).